<a href="https://colab.research.google.com/github/NAVEED261/MY-AI-ASSISTANT/blob/main/database_with_tool_cal.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
%%capture --no-stderr
%pip install -U langgraph langgraph-checkpoint-postgres psycopg psycopg-pool langchain_google_genai

from google.colab import userdata
GEMINI_API_KEY = userdata.get('GOOGLE_API_KEY')

import os
os.environ["LANGCHAIN_API_KEY"] = userdata.get('Langchain_api_key')
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_PROJECT"] = "langchain-academy"

from psycopg_pool import ConnectionPool
from langgraph.checkpoint.postgres import PostgresSaver

# Set up PostgreSQL Database
DB_URI = userdata.get('DB_URI')
connection_kwargs = {"autocommit": True, "prepare_threshold": 0}
pool = ConnectionPool(conninfo=DB_URI, max_size=20, kwargs=connection_kwargs)
checkpointer = PostgresSaver(pool)
checkpointer.setup()

# Define Tools
from langchain_core.tools import Tool

# Tool 1: Fetch Product Details
def get_product_details(product_id: str) -> str:
    products = {
        "P101": {"name": "Laptop", "price": 10600.0, "stock": 5},
        "P102": {"name": "Smartphone", "price": 7500.0, "stock": 0},
    }
    product = products.get(product_id)
    if not product:
        return "Product not found."
    return f"Name: {product['name']}, Price: ${product['price']}, Stock: {product['stock']} units."

# Tool 2: Calculate Shipping Cost
def calculate_shipping_cost(distance: float, weight: float) -> float:
    rate_per_km_per_kg = 5.0
    return distance * weight * rate_per_km_per_kg

# Register Tools
tools = [
    Tool(name="get_product_details", func=get_product_details, description="Fetch product details by ID."),
    Tool(name="calculate_shipping_cost", func=calculate_shipping_cost, description="Calculate shipping cost."),
]

# Initialize Model with Tools
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", api_key=GEMINI_API_KEY)
llm_with_tools = model.bind_tools(tools)

# Define Custom State
from langchain_core.messages import SystemMessage, HumanMessage, RemoveMessage
from langgraph.graph import StateGraph, START, END, MessagesState
from langgraph.prebuilt import ToolNode
from IPython.display import Image

class State(MessagesState):
    summary: str

# Model Logic
def call_model_with_tools(state: State) -> State:
    messages = state["messages"]
    response = llm_with_tools.invoke(messages)
    return {"messages": response}

# Summarize Conversation
def summarize_conversation(state: State) -> State:
    summary = state.get("summary", "")
    summary_message = (
        f"Extend the summary by taking into account the new messages above:\n{summary}"
        if summary else "Create a summary of the conversation above:"
    )
    messages = state["messages"] + [HumanMessage(content=summary_message)]
    response = model.invoke(messages)
    delete_messages = [RemoveMessage(id=m.id) for m in state["messages"][:-2]]
    return {"summary": response.content, "messages": delete_messages}

# Workflow Logic
def should_continue(state: State) -> str:
    if len(state["messages"]) > 6:
        return "summarize_conversation"
    return END

# Define Graph
workflow = StateGraph(State)
workflow.add_node("conversation", call_model_with_tools)
workflow.add_node("tools", ToolNode(tools))
workflow.add_node("summarize_conversation", summarize_conversation)
workflow.add_edge(START, "conversation")
workflow.add_edge("conversation", "tools")
workflow.add_edge("tools", "summarize_conversation")
workflow.add_edge("summarize_conversation", END)

# Compile Workflow with Database Checkpointer
graph = workflow.compile(checkpointer=checkpointer)
display(Image(graph.get_graph().draw_mermaid_png()))

In [5]:
config = {"configurable": {"thread_id": "3"}}

# Start conversation
input_message = HumanMessage(content="Show me the details of product P101")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages']:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  get_product_details (9606206f-762d-475e-8cc2-3649a283f26c)
 Call ID: 9606206f-762d-475e-8cc2-3649a283f26c
  Args:
    __arg1: P101
================================= Tool Message =================================
Name: get_product_details

Name: Laptop, Price: $10600.0, Stock: 5 units.


In [6]:
input_message = HumanMessage(content="What is the shipping cost for 100 km and 2 kg?")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages']:
    m.pretty_print()

================================== Ai Message ==================================
Tool Calls:
  calculate_shipping_cost (8cdd3138-6023-486b-a9c4-e51adcc66b61)
 Call ID: 8cdd3138-6023-486b-a9c4-e51adcc66b61
  Args:
    __arg1: 100km,2kg
================================= Tool Message =================================
Name: calculate_shipping_cost

Error: TypeError("calculate_shipping_cost() missing 1 required positional argument: 'weight'")
 Please fix your mistakes.


In [7]:
input_message = HumanMessage(content="I want to buy a laptop (P101). Can you confirm its price, availability, calculate the shipping cost for 15 km with a weight of 2.5 kg, and also add a review saying, 'Great performance!'?")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages']:
    m.pretty_print()

================================ Human Message =================================

I want to buy a laptop (P101). Can you confirm its price, availability, calculate the shipping cost for 15 km with a weight of 2.5 kg, and also add a review saying, 'Great performance!'?
================================== Ai Message ==================================

I cannot fulfill this request completely. I can get the price and availability of the laptop if you provide the necessary API calls or data structures.  I also cannot add a review as there is no such functionality available in the provided APIs.  I can calculate the shipping cost if the `calculate_shipping_cost` function is updated to accept distance and weight as parameters.  The current function is not usable.


In [8]:
input_message = HumanMessage(content="Calculates the shipping cost for 15 km and 2.5 kg.")
output = graph.invoke({"messages": [input_message]}, config)
for m in output['messages']:
    m.pretty_print()

================================ Human Message =================================

Calculates the shipping cost for 15 km and 2.5 kg.
================================== Ai Message ==================================

I cannot perform this calculation. The available function `calculate_shipping_cost` does not accept distance and weight as parameters.  To calculate the shipping cost, I need either a function that takes distance and weight as input or a pricing model.
